# Deep Convolutional Generative Adversarial Network (DCGAN)

In [6]:
import os
import sys
import datetime as dt

import numpy as np
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt

from dataset import ImageDataset

%matplotlib inline

### Loding dataset

In [2]:
data = ImageDataset(data_dir='datasets/flowers/', size=50, flatten=False, grayscale=False)
# data.create()
# data.save('datasets/saved/data.pkl')
# data = data.load('datasets/saved/data.pkl')

Processing 5 of 5 class labels & 799 of 799 images

In [4]:
X_train, _, X_test, _ = data.train_test_split(test_size=0.1)
print(X_train.shape, X_test.shape)

(3303, 50, 50, 3) (367, 50, 50, 3)


In [5]:
image_size = data.size
image_channel = data.channels
image_size_flat = image_size * image_size * image_channel
print('size: {}\tchannel: {}\t flattened: {:,}'.format(image_size, image_channel, image_size_flat))

size: 50	channel: 3	 flattened: 7,500


In [11]:
keep_prob = 0.8

### Discriminator `(Deep Convolutional Neural Net)`

In [16]:
def discriminator(image, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Building 'AlexNet'
        network = tf.reshape(image, shape=[-1, image_size, image_size, image_channel])
        # 1st convnet
        network = tflearn.conv_2d(network, nb_filter=96, filter_size=11, strides=4, activation='relu',)
        network = tflearn.max_pool_2d(network, kernel_size=3, strides=2)
        network = tflearn.batch_normalization(network)
        # 2nd convnet
        network = tflearn.conv_2d(network, nb_filter=256, filter_size=5, activation='relu')
        network = tflearn.max_pool_2d(network, kernel_size=3, strides=2)
        network = tflearn.batch_normalization(network)
        # 3, 4, 5 convnet
        network = tflearn.conv_2d(network, nb_filter=384, filter_size=3, activation='relu')
        network = tflearn.conv_2d(network, nb_filter=384, filter_size=3, activation='relu')
        network = tflearn.conv_2d(network, nb_filter=256, filter_size=3, activation='relu')
        network = tflearn.max_pool_2d(network, kernel_size=3, strides=2)
        network = tflearn.batch_normalization(network)
        # 1st fully connected
        network = tflearn.fully_connected(network, n_units=4096, activation='tanh')
        network = tflearn.dropout(network, keep_prob)
        # 2nd fully connected
        network = tflearn.fully_connected(network, n_units=4096, activation='tanh')
        network = tflearn.dropout(network, keep_prob)
        # 3rd fully connected
        network = tflearn.fully_connected(network, n_units=1, activation='softmax')
    return network

## Generator `(Deep Deconvolutional Neural Net)`

In [56]:
def generator(noise, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        x = tflearn.fully_connected(noise, n_units=4096, activation='tanh')
        x = tflearn.batch_normalization(x)
        x = tflearn.fully_connected(x, n_units=1024, activation='tanh')  # 8*8*16=1024
        x = tf.reshape(x, shape=[-1, 8, 8, 16])
        x = tflearn.upsample_2d(x, 2)
        x = tflearn.conv_2d(x, nb_filter=256, filter_size=5, activation='relu')
        x = tflearn.conv_2d(x, nb_filter=384, filter_size=5, activation='relu')
        x = tflearn.conv_2d(x, nb_filter=384, filter_size=5, activation='relu')
        x = tflearn.batch_normalization(x)
        x = tflearn.upsample_2d(x, kernel_size=2)
        x = tflearn.conv_2d(x, nb_filter=256, filter_size=5, activation='relu')
        x = tflearn.batch_normalization(x)
        x = tflearn.conv_2d(x, nb_filter=image_size_flat, filter_size=5, activation='relu')
        x = tflearn.upsample_2d(x, kernel_size=2)
        x = tflearn.conv_2d(x, nb_filter=96, filter_size=5, activation='relu')
        x = tf.reshape(x, shape=[-1, image_size, image_size, image_channel])
        return x

In [57]:
z_dim = 200

tf.reset_default_graph()

gen_input = tflearn.input_data(shape=[None, z_dim], name='input_noise')
disc_input = tflearn.input_data(shape=[None, image_size_flat], name='disc_input')

gen_sample = generator(gen_input)
disc_real = discriminator(disc_input)
disc_fake = discriminator(gen_sample, reuse=True)

Tensor("discriminator_1/FullyConnected_2/Softmax:0", shape=(?, 1), dtype=float32)
